In [ ]:
import os
import sys
import random
import warnings
import io 

import numpy as np
import pandas as pd
# import matplotlib
# import matplotlib.pyplot as plt
# plt.style.use("ggplot")

# # matplotlib.use('agg')
import matplotlib.pyplot as plt
%matplotlib inline

# OK
from tqdm import tqdm
from itertools import chain
from skimage.io import imread, imshow, imread_collection, concatenate_images
from skimage.transform import resize
from skimage.morphology import label

from keras.models import Model, load_model
from keras.layers import Input
from keras.layers.core import Dropout, Lambda
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K

import tensorflow as tf
import cv2
import matplotlib.pyplot as plt
from skimage.color import rgb2gray
from skimage import data
from google.colab.patches import cv2_imshow
print("Imported Tensorflow",tf.__version__)
from tensorflow.keras.optimizers import Adam

In [ ]:
# OK
# Set some parameters
IMG_WIDTH = 128
IMG_HEIGHT = 128
IMG_CHANNELS = 3
TRAIN_PATH = '/content/drive/MyDrive/U-Net-3/data/train/'
TEST_PATH = '/content/drive/MyDrive/U-Net-3/data/test/'

warnings.filterwarnings('ignore', category=UserWarning, module='skimage')
seed = 42
random.seed = seed
np.random.seed = seed

print("Imported all the dependencies")

In [ ]:
# OK
# le chemin pour les images et leurs masques dans le dossier train
trainImagePath = os.path.sep.join([TRAIN_PATH,'images'])
trainMaskPath = os.path.sep.join([TRAIN_PATH,'masks'])


# Get train and test IDs
train_ids = os.listdir(trainImagePath)
mask_ids = os.listdir(trainMaskPath)
"""for e in train_ids :
  if e not in mask_ids :
    print(e)"""

#test_ids = os.listdir(testImagePath)
# Get and resize train images and masks
X_train = np.zeros((len(train_ids), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
#Y_train = np.zeros((len(train_ids), IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.bool)
Y_train = np.zeros((len(train_ids), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)

print("X_train",X_train.shape)
print("Y_train",Y_train.shape)

print('Getting and resizing train images and masks ... ')
sys.stdout.flush()
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

for n,idsImage in enumerate(train_ids):
  mask = np.zeros((IMG_HEIGHT, IMG_WIDTH, 3), dtype=np.uint8)
  #mask = np.zeros((IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.uint8)
  pathimage = os.path.sep.join([trainImagePath,idsImage])
  img = imread(pathimage, plugin='matplotlib')[:,:,:IMG_CHANNELS]
  img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
  X_train[n] = img
  pathmask = os.path.sep.join([trainMaskPath,idsImage])
  #mask_ = rgb2gray(imread(pathmask))
  mask_ = imread(pathmask)
  #mask_ = imread(path + '/masks/' + mask_file)
  #mask_ = np.expand_dims(resize(mask_, (IMG_HEIGHT, IMG_WIDTH), mode='constant',preserve_range=True), axis=-1)
  mask_ = resize(mask_, (IMG_HEIGHT, IMG_WIDTH), mode='constant',preserve_range=True)
  mask = np.maximum(mask, mask_)
  Y_train[n] = mask
  print("image",n,"path",idsImage)



#for n,mask_file in enumerate(os.listdir(trainMaskPath)):
  
#print("mask",n)
# Get and resize test images
#X_test = np.zeros((len(test_ids), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
#sizes_test = []
#print('Getting and resizing test images ... ')
#sys.stdout.flush()
#for n,idsTest in enumerate(test_ids):
  #path = os.path.sep.join([test_ids,idsTest])
  #img = imread(path)[:,:,:IMG_CHANNELS]
  #sizes_test.append([img.shape[0], img.shape[1]])
  #img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
  #X_test[n] = img
  #print("test",n)

print('Done!')

In [ ]:
# OK
testImagePath = os.path.sep.join([TEST_PATH,'images'])


test_ids = os.listdir(testImagePath)


X_test = np.zeros((len(test_ids), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
Y_test = np.zeros((len(test_ids), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)

sizes_test = []
print('Getting and resizing test images ... ')
sys.stdout.flush()
for n,idsTest in enumerate(test_ids):
  path = os.path.sep.join([testImagePath,idsTest])
  img = imread(path)[:,:,:IMG_CHANNELS]
  sizes_test.append([img.shape[0], img.shape[1]])
  img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
  X_test[n] = img
  print("test",n)

In [ ]:
# OK
# Check if training data looks all right
ix = random.randint(0, len(train_ids))

imshow(X_train[ix])
plt.savefig('img.jpg')
plt.show();


#plt.imshow(Y_train[ix][:,:,0],cmap=plt.cm.gray)
#plt.imshow(Y_train[ix],cmap=plt.cm.gray)
#plt.imshow(Y_train[ix][:,:,0])
plt.imshow(Y_train[ix])
plt.savefig('mask.jpg')
plt.show();


In [ ]:
# OK
# Cette fonction n'est pas compatible avec tes données
#Elle  
# Define IoU metric
def mean_iou(y_true, y_pred):
    prec = []
    for t in np.arange(0.5, 1.0, 0.05):
        y_pred_ = tf.cast(y_pred > t,tf.int32)
        score, up_opt = tf.metrics.MeanIou(y_true, y_pred_, 2)
        K.get_session().run(tf.local_variables_initializer())
        with tf.control_dependencies([up_opt]):
            score = tf.identity(score)
        prec.append(score)
    return K.mean(K.stack(prec), axis=0)

In [ ]:
# OK
# Build U-Net model
inputs = Input((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
s = Lambda(lambda x: x / 255) (inputs)

c1 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (s)
c1 = Dropout(0.1) (c1)
c1 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c1)
p1 = MaxPooling2D((2, 2)) (c1)

c2 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p1)
c2 = Dropout(0.1) (c2)
c2 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c2)
p2 = MaxPooling2D((2, 2)) (c2)

c3 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p2)
c3 = Dropout(0.2) (c3)
c3 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c3)
p3 = MaxPooling2D((2, 2)) (c3)

c4 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p3)
c4 = Dropout(0.2) (c4)
c4 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c4)
p4 = MaxPooling2D(pool_size=(2, 2)) (c4)

c5 = Conv2D(256, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p4)
c5 = Dropout(0.3) (c5)
c5 = Conv2D(256, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c5)

u6 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same') (c5)
u6 = concatenate([u6, c4])
c6 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u6)
c6 = Dropout(0.2) (c6)
c6 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c6)

u7 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same') (c6)
u7 = concatenate([u7, c3])
c7 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u7)
c7 = Dropout(0.2) (c7)
c7 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c7)

u8 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same') (c7)
u8 = concatenate([u8, c2])
c8 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u8)
c8 = Dropout(0.1) (c8)
c8 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c8)

u9 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same') (c8)
u9 = concatenate([u9, c1], axis=3)
c9 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u9)
c9 = Dropout(0.1) (c9)
c9 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c9)

#outputs = Conv2D(1, (1, 1), activation='sigmoid') (c9)
outputs = Conv2D(3, (1, 1), activation='sigmoid') (c9)

model = Model(inputs=[inputs], outputs=[outputs])
model.compile(optimizer=Adam(learning_rate = 1e-3), loss='binary_crossentropy', metrics=['accuracy'])
model.summary()
print("The model is defined")

In [ ]:
# OK
# print(X_train)
# print(Y_train)#
history = model.fit(X_train, Y_train,
                    batch_size=16,
                    verbose=1,
                    epochs=16,
                    validation_split=0.2,
                    shuffle=False)


In [ ]:
# OK
model.save('/content/drive/MyDrive/Unet-8/model/model-dsbowl2018-1(1).h5')


In [ ]:
# OK
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'y', label='Training loss' )
plt.plot(epochs, val_loss, 'r', label='Validation loss' )
plt.legend(loc='lower right')
plt.title('Training and Validation loss')
plt.xlabel('epochs')

In [ ]:
# OK
loss = history.history['accuracy']
val_loss = history.history['val_accuracy']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'y', label='Training accuracy' )
plt.plot(epochs, val_loss, 'r', label='Validation accuracy' )
plt.legend(loc='lower right')
plt.title('Training and Validation accuracy')
plt.xlabel('epochs')

In [ ]:
# OK
# Predict on train, val and test

preds_train = model.predict(X_train[:int(X_train.shape[0]*0.9)], verbose=1)
preds_val = model.predict(X_train[int(X_train.shape[0]*0.9):], verbose=1)
preds_test = model.predict(X_test, verbose=1)

# Threshold predictions
preds_train_t = (preds_train > 0.5).astype(np.uint8)
preds_val_t = (preds_val > 0.5).astype(np.uint8)
preds_test_t = (preds_test > 0.5).astype(np.uint8)

In [ ]:
# OK
# Create list of upsampled test masks
preds_test_upsampled = []
for i in range(len(preds_test)):
    preds_test_upsampled.append(resize(np.squeeze(preds_test[i]), 
                                       (sizes_test[i][0], sizes_test[i][1]), 
                                       mode='constant', preserve_range=True))


**TEST U-NET**

** ================================= Debut =================================**

In [ ]:
import os
import sys
import random
import warnings

import numpy as np
import pandas as pd
import matplotlib
matplotlib.use('agg')
import matplotlib.pyplot as plt


from tqdm import tqdm
from itertools import chain
from skimage.io import imread, imshow, imread_collection, concatenate_images
from skimage.transform import resize
from skimage.morphology import label

from keras.models import Model, load_model
from keras.layers import Input
from keras.layers.core import Dropout, Lambda
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K

import tensorflow as tf
import cv2


In [ ]:
IMG_WIDTH = 128
IMG_HEIGHT = 128
IMG_CHANNELS = 3
# TRAIN_PATH = '/content/drive/MyDrive/U-Net-8/data/train/'
TEST_PATH = '/content/drive/MyDrive/U-Net-8/data/test/'

**Automatic map labelling**

In [ ]:
class automaticmaplabelling():
    def __init__(self,modelPath,full_chq,imagePath,width,height,channels):
        print (modelPath)
        print(imagePath) 
        print(width)
        print(height)
        print(channels)
        self.modelPath=modelPath
        self.full_chq=full_chq
        self.imagePath=imagePath
        self.IMG_WIDTH=width
        self.IMG_HEIGHT=height
        self.IMG_CHANNELS=channels
        self.model = U_net()

**mean_iou**

In [ ]:
def mean_iou(self,y_true, y_pred):
        prec = []
        for t in np.arange(0.5, 1.0, 0.05):
            y_pred_ = tf.to_int32(y_pred > t)
            score, up_opt = tf.metrics.mean_iou(y_true, y_pred_, 2)
            K.get_session().run(tf.local_variables_initializer())
            with tf.control_dependencies([up_opt]):
                score = tf.identity(score)
            prec.append(score)
        return K.mean(K.stack(prec), axis=0)

In [ ]:
# Build U-Net model
def U_net():
  inputs = Input((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
  s = Lambda(lambda x: x / 255) (inputs)

  c1 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (s)
  c1 = Dropout(0.1) (c1)
  c1 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c1)
  p1 = MaxPooling2D((2, 2)) (c1)

  c2 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p1)
  c2 = Dropout(0.1) (c2)
  c2 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c2)
  p2 = MaxPooling2D((2, 2)) (c2)

  c3 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p2)
  c3 = Dropout(0.2) (c3)
  c3 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c3)
  p3 = MaxPooling2D((2, 2)) (c3)

  c4 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p3)
  c4 = Dropout(0.2) (c4)
  c4 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c4)
  p4 = MaxPooling2D(pool_size=(2, 2)) (c4)

  c5 = Conv2D(256, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p4)
  c5 = Dropout(0.3) (c5)
  c5 = Conv2D(256, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c5)

  u6 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same') (c5)
  u6 = concatenate([u6, c4])
  c6 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u6)
  c6 = Dropout(0.2) (c6)
  c6 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c6)

  u7 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same') (c6)
  u7 = concatenate([u7, c3])
  c7 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u7)
  c7 = Dropout(0.2) (c7)
  c7 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c7)

  u8 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same') (c7)
  u8 = concatenate([u8, c2])
  c8 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u8)
  c8 = Dropout(0.1) (c8)
  c8 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c8)

  u9 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same') (c8)
  u9 = concatenate([u9, c1], axis=3)
  c9 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u9)
  c9 = Dropout(0.1) (c9)
  c9 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c9)


  outputs = Conv2D(3, (1, 1), activation='sigmoid') (c9)

  model = Model(inputs=[inputs], outputs=[outputs])

# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[mean_iou, 'accuracy'])
  model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
  modelPath = ('/content/drive/MyDrive/Unet-8/model/model-dsbowl2018-1(1).h5')
  model.load_weights(modelPath)
  model.summary()
  return model

**Prediction**

In [ ]:
# Perform a sanity check on some random testing samples
#Y_test = np.zeros((len(test_ids), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
from google.colab.patches import cv2_imshow
ix = random.randint(0, len(preds_test_t))
imshow(X_test[ix])
plt.savefig('img.png')
plt.show();
def prediction():
  # img=cv2.imread(imagePath,0)
  img=X_test[ix]
  # img=np.expand_dims(img,axis=-1)
  x_test= np.zeros((1, IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
  #testimg=resize(img,(self.IMG_HEIGHT,self.IMG_WIDTH),mode='constant',preserve_range=True)
  x_test[0]=img
  preds_test= model.predict(x_test, verbose=1)

  preds_test = (preds_test > 0.5).astype(np.uint8)
  mask=preds_test[0]
  print(mask)
  """print (mask.shape[0])
  print (mask.shape[1])
  for i in range(mask.shape[0]):
    for j in range(mask.shape[1]):
      print(mask[i][j])
      if mask[i][j] == 1:
        mask[i][j] = 255
      else:
        mask[i][j] = 0"""
  merged_image = cv2.merge((mask,mask,mask))
  #cv2_imshow(merged_image)
  mask = cv2.cvtColor(mask, cv2.COLOR_RGB2GRAY)
  contours, hierarchy = cv2.findContours(mask,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)

  for each_contour in contours:
    x,y,w,h = cv2.boundingRect(each_contour)

  # cv2.rectangle(merged_image,(x,y),(x+w,y+h),(0,0,255),4)
    cv2.rectangle(mask,(x,y),(x+w,y+h),(0,0,255),4)

    print (x,y,w,h)
  #cv2.imshow("merged_image",merged_image)
    cv2_imshow(mask)
    cv2.waitKey(0)
        
  cv2.imwrite("mask.png",mask)
  return x_test[0],mask


**Programme Principal**

In [ ]:
test_image_name = "/content/drive/MyDrive/U-Net-8/data/test1/0012_2m_0P_-10V_0H.jpg"
#automaticmaplabellingobj= automaticmaplabelling('model-dsbowl2018-1.h5',True,test_image_name,128,128,3)
automaticmaplabellingobj= automaticmaplabelling('model-dsbowl2018-1.h5',True,test_image_name,128,128,3)
testimg,mask = prediction()
#mask = prediction()
print('Showing images..')
#cv2_imshow('img',testimg)
dim = (1280, 1280)
resized = cv2.resize(mask, dim, interpolation = cv2.INTER_AREA)
    
cv2_imshow(mask)
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.imwrite("resized_mask.png",resized)

 ** ================================= FIN =================================**

In [ ]:
# OK
testImagePath = os.path.sep.join([TEST_PATH,'images'])

test_ids = os.listdir(testImagePath)

X_test = np.zeros((len(test_ids), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
Y_test = np.zeros((len(test_ids), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)

sizes_test = []
print('Getting and resizing test images ... ')
sys.stdout.flush()
for n,idsTest in enumerate(test_ids):
  path = os.path.sep.join([testImagePath,idsTest])
  img = imread(path)[:,:,:IMG_CHANNELS]
  sizes_test.append([img.shape[0], img.shape[1]])
  img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
  X_test[n] = img
  print("test",n)
  cv2_imshow(X_test[n])
  # cv2_imshow(img)

# ix = random.randint(0, len(preds_test_t))
# imshow(X_test[ix])
# plt.savefig('img.png')
# plt.show();


img=imread(path)[:,:,:IMG_CHANNELS]
cv2_imshow(img)
img=np.expand_dims(img,axis=-1)
x_test= np.zeros((1, IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
testimg=resize(img,(IMG_HEIGHT,IMG_WIDTH),mode='constant',preserve_range=True)
x_test[0]=img
# preds_test= model.predict(x_test, verbose=1)
        
# preds_test = (preds_test > 0.5).astype(np.uint8)
# mask=preds_test[0]
# for i in range(mask.shape[0]):
#   for j in range(mask.shape[1]):
#     if mask[i][j] == 1:
#       mask[i][j] = 255
#     else:
#       mask[i][j] = 0

# merged_image = cv2.merge((mask,mask,mask))
# contours, hierarchy = cv2.findContours(mask,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
# for each_contour in contours:
#   x,y,w,h = cv2.boundingRect(each_contour)

#   cv2.rectangle(merged_image,(x,y),(x+w,y+h),(0,0,255),4)

#   print (x,y,w,h)
#   cv2.imshow("merged_image",merged_image)
#   cv2.waitKey(0)
        
# cv2.imwrite("mask.png",mask)

# return x_test[0],mask

In [ ]:
# # prediction
# ix = random.randint(0, len(preds_test_t))
# imshow(X_test[ix])
# plt.savefig('img.png')
# plt.show();


# # img=cv2.imread(self.imagePath,0)
        
        
# #         img=np.expand_dims(img,axis=-1)
# #         x_test= np.zeros((1, self.IMG_HEIGHT, self.IMG_WIDTH, self.IMG_CHANNELS), dtype=np.uint8)
# #         #testimg=resize(img,(self.IMG_HEIGHT,self.IMG_WIDTH),mode='constant',preserve_range=True)
# #         x_test[0]=img
# #         preds_test= self.model.predict(x_test, verbose=1)
        
# #         preds_test = (preds_test > 0.5).astype(np.uint8)
# #         mask=preds_test[0]
# #         for i in range(mask.shape[0]):
# #             for j in range(mask.shape[1]):
# #                 if mask[i][j] == 1:
# #                     mask[i][j] = 255
# #                 else:
# #                     mask[i][j] = 0
# #         merged_image = cv2.merge((mask,mask,mask))
# #         contours, hierarchy = cv2.findContours(mask,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
# #         for each_contour in contours:
# #             x,y,w,h = cv2.boundingRect(each_contour)

# #             cv2.rectangle(merged_image,(x,y),(x+w,y+h),(0,0,255),4)

# #             print (x,y,w,h)
# #             cv2.imshow("merged_image",merged_image)
# #             cv2.waitKey(0)
        
# #         cv2.imwrite("mask.png",mask)

# #         # return x_test[0],mask


In [ ]:
testImagePath = os.path.sep.join([TEST_PATH,'images'])


test_ids = os.listdir(testImagePath)


#X_test = np.zeros((len(test_ids), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
Y_test = np.zeros((len(test_ids), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)

sizes_test = []
print('Getting and resizing test images ... ')
#sys.stdout.flush()
for n,idsTest in enumerate(test_ids):
  #path = os.path.sep.join([testImagePath,idsTest])
  img=preds_test[n]
  #img = imread(path)[:,:,:IMG_CHANNELS]
  #sizes_test.append([img.shape[0], img.shape[1]])
  #img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
  Y_test[n] = img
  print("test",n)

**PREDICTION**

In [ ]:
# # Perform a sanity check on some random testing samples
# #Y_test = np.zeros((len(test_ids), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
# ix = random.randint(0, len(preds_test_t))
# imshow(X_test[ix])
# plt.savefig('img.png')
# plt.show();
# #preds_test[ix]
# #Y_test=preds_test[ix]
# #print(preds_test[ix])
# #imshow( Y_test[ix])
# #imshow(np.squeeze(preds_test[ix]))
# #plt.savefig('gt.png')
# #plt.show();

# #test=Y_test[ix]
# #imshow(np.squeeze(Y_test[ix]))

# #cv2.imwrite('gt.png',test)
# #imshow('gt.png')
# #plt.show();"""

# """image =imread('/content/gt.png')
# im_bw = cv2.Canny(image, 255, 255)
# contours, hierarchy = cv2.findContours(im_bw, mode=cv2.RETR_TREE, method=cv2.CHAIN_APPROX_NONE)
# # draw contours on the original image
# image_copy = image.copy()
# cv2.drawContours(image=image_copy,contours=contours,  contourIdx=-1, color=(0, 255, 0), thickness=2, lineType=cv2.LINE_AA)
# # see the results
# cv2_imshow(image_copy)
# cv2.waitKey(0)
# cv2.imwrite('contours_none_image1.png', image_copy)
# cv2.destroyAllWindows()"""


In [ ]:
# # detect the contours on the binary image using cv2.CHAIN_APPROX_NONE
# from google.colab.patches import cv2_imshow
# from skimage.io import imread
# import cv2 
# im_bw =imread('/content/gt.PNG')
# im_bw = cv2.Canny(im_bw, 255, 255)
# contours, hierarchy = cv2.findContours(im_bw, mode=cv2.RETR_TREE, method=cv2.CHAIN_APPROX_NONE)
                                     
# # draw contours on the original image
# #image_copy = image.copy()
# cv2.drawContours(image=im_bw, contours=contours, contourIdx=-1, color=(0, 255, 0), thickness=2, lineType=cv2.LINE_AA)
               
# # see the results
# cv2_imshow(im_bw)
# cv2.waitKey(0)
# cv2.imwrite('contours_none_image1.jpg', im_bw)
# cv2.destroyAllWindows()